In [1]:
file = open("../../NightSix/obs.txt", "r")
#file = open("../../NightSix/reg.txt", "r")
obs = [line for line in file]
file.close()


In [2]:
def parse_hl7(file_lines):
    delimiters = file_lines[0].split("|")[1]

    json_dict = dict()

    for i, line in enumerate(file_lines):
        

        #Key for the message header will be the line number (i) \. the first field in the row
        #if less than 10, add padding so it can be sorted correctly
        if i < 10:
            msh = "0"+str(i) + "." + line.split("|")[0]
        else:
            msh = str(i) + "." + line.split("|")[0]

        json_dict[msh] = {}
        line = line.replace("\n", "")

        for ii, field in enumerate(line.split("|")):
            
            #construct key with MESSAGE HEADER.i.ii
            key = msh+"."+str(ii)


            # don't want the field if it is null
            if len(field)==0 or field=="":
                continue

            #dictionaries with nested keys/values need to be initialized before we can use them
            json_dict[msh][key] = {}

            for iii, segment in enumerate(field.split(delimiters[0])):
                #Construct a segment key with our previous key with an additional numeric value
                key_segment = key+"."+str(iii)

                #don't want the field if it is null
                if len(segment)==0 or segment=="":
                    continue

                #dictionaries with nested keys/values need to be initialized before we can use them
                json_dict[msh][key][key_segment] = {}

                for iiii, component in enumerate(segment.split(delimiters[1])):
                    #construct a key for our compoments
                    key_component = key_segment+"."+str(iiii)

                    """ inject the values by assigning the most inner values first then going outwards """

                    #only need to assign the inner dictionary value if it exists 
                    #(if we split on the delimiter and more than one value exists)
                    if len(segment.split("~")) > 1: 

                        #don't want the field if it is null
                        if len(component)==0 or component=="":
                            continue
                        json_dict[msh][key][key_segment][key_component] = component
                    elif len(field.split("^")) > 1:

                         #don't want the field if it is null
                        if len(segment)==0 or segment=="":
                            continue
                        json_dict[msh][key][key_segment] = segment
                    else:

                         #don't want the field if it is null
                        if len(segment)==0 or segment=="":
                            continue
                        json_dict[msh][key] = field
                        
        #awful way of fixing the delimiter parsing for the second field
        json_dict["00.MSH"]["00.MSH.1"] = delimiters

    return json_dict


In [3]:
json_dict = parse_hl7(obs)

In [4]:
from collections import OrderedDict
def reorderDictItems(my_dictionary):
    return OrderedDict(my_dictionary)

In [5]:
#dictionaries aren't ordered. Reorder the keys
for key in json_dict.keys():
    json_dict[key] = reorderDictItems(json_dict[key])
    for inner_key in json_dict[key].keys():
        #print(key, inner_key)
        if type(json_dict[key][inner_key])==dict:
            json_dict[key][inner_key] = reorderDictItems(json_dict[key][inner_key])

In [6]:
import json
def write_json(my_dictionary, outfile):
    my_json = json.dumps(my_dictionary)
    file = open(outfile, "w")
    file.write(my_json)
    file.close()

In [7]:
write_json(json_dict, "my_json.json")

In [8]:
cat my_json.json | pbcopy